In [1]:
# Single Layer LSTM Long Short-Term Memory
# RNN takes the ordering of inputs into account
# parts-of-speech tagging, music composition, language translation
# model to differentiate sentiments even if the words used in two sentences are the same
# 1. My friends do like the cake but I don't -> NEGATIVE REVIEW
# 2. My friends don't like the cake but I do -> POSITIVE REVIEW

# LSTM computes the state of a current timestep and passes it on to the next timesteps where this
# state is also updated. The process repeats until the final timestep where the output computation
# is affected by all previous states. It can be configured to be bidirectional in order to get the
# relationship of later words to earlier ones.

import tensorflow_datasets as tfds

# Download the IMBD review subwords 8k
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

# Get the tokenizer
tokenizer = info.features['text'].encoder

In [2]:
# Prepare the dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 256

# Get the train and test splits
train_data, test_data = dataset['train'], dataset['test'],

# Shuffle the training data
train_dataset = train_data.shuffle(BUFFER_SIZE)

# Batch and pad the datasets to the maximum length of the sequences
train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_data.padded_batch(BATCH_SIZE)

In [3]:
# Build and compile the model
# Swap Flatten or GlobalAveragePooling1D for LSTM

import tensorflow as tf

# Hyperparameters
embedding_dim = 64
lstm_dim = 64
dense_dim = 64

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
NUM_EPOCHS = 2#10
history = model.fit(train_dataset, epochs=NUM_EPOCHS, validation_data=test_dataset, verbose=2)

Epoch 1/10
98/98 - 5760s - loss: 0.5888 - accuracy: 0.6714 - val_loss: 0.4602 - val_accuracy: 0.7976
Epoch 2/10


In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history,"accuracy")
plot_graphs(history,"loss")